In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [2]:
data = pd.read_csv('data/adult.csv')

Признаки:
* age - возраст
* workclass - статус занятости
* fnlwgt - общий вес, это число людей, которых, по мнению эксперта, представляет эта категория занятости
* education - образование
* education.num - образовательная ступень (в виде числа)
* marital.status - брачный статус
* occupation - профессия
* relationship - тип отношений
* race - раса
* sex - пол
* capital.gain - ежегодный дополнительный прирост капитала
* capital.loss - ежегодная дополнительная потеря капитала
* hours.per.week - число рабочих часов в неделю
* native.country - Родина 
* income - категория дохода (целевой признак)

In [3]:
data['income'] = data['income'].apply(lambda x: 1 if x == '>50K' else 0)

types = data.dtypes
cat_features = list(types[(types == 'object')].index)
cat_features

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [4]:
adult_data = data.copy()
#Определяем школьников в отдельную категорию
adult_data['education'] = adult_data['education'].apply(
    lambda x: 'School' if x == '11th' or x == '7th-8th' or x == '10th' 
        or x == '5th-6th' or x == '9th' or x == '12th' or x == '1st-4th' else x
)
#Объединяем категории Assoc-acdm и Assoc-voc (доценты)
adult_data['education'] = adult_data['education'].apply(
    lambda x: 'Associate' if x == 'Assoc-acdm' or x == 'Assoc-voc' else x
)
#Объединяем вдовцов,разведенных и живущих раздельно в одну категорию
adult_data['marital.status'] = adult_data['marital.status'].apply(
    lambda x: 'Prev-Married' if (x == 'Widowed' or x == 'Divorced' or x == 'Separated') else x
)
#Объединяем всех женатых/за мужем в одну категорию
adult_data['marital.status'] = adult_data['marital.status'].apply(
    lambda x: 'Married' if (x == 'Married-civ-spouse' or x == 'Married-spouse-absent' or x == 'Married-AF-spouse') else x
)
#Объединяем мужей и жен в одну категорию, остальных в другую
adult_data['relationship'] = adult_data['relationship'].apply(
    lambda x: 'In relationship' if (x == 'Husband' or x == 'Whife') else 'Not in relationship'
)

#Объединяем типы занятоностей, не приносящих дохода в одну категорию
adult_data['workclass'] = adult_data['workclass'].apply(
    lambda x: 'No income' if x == 'Never-worked' or x == 'Without-pay' else x
)
#Объединяем всех приезжих в одну категорию
adult_data['native.country'] = adult_data['native.country'].apply(
    lambda x: 'Other' if x != 'United-States' else x
)

In [5]:
#Логарифмируем числовые признаки, чтобы придать им форму нормального распределения
adult_data['capital.gain'] = np.log(adult_data['capital.gain']+1)
adult_data['capital.loss'] = np.log(adult_data['capital.loss']+1)
adult_data['fnlwgt'] = np.log(adult_data['fnlwgt']+1)

#Создаем новый признак - разность между приростом капитала и его убылью
adult_data['capital_diff'] = abs((adult_data['capital.gain'] - adult_data['capital.loss']))

#Удаляем лишние признаки
adult_data = adult_data.drop(['education.num', 'capital.gain', 'capital.loss'], axis=1)

dummies_data = pd.get_dummies(adult_data, drop_first=True)
dummies_data.head()

,age,fnlwgt,hours.per.week,income,capital_diff,workclass_Federal-gov,workclass_Local-gov,workclass_No income,workclass_Private,workclass_Self-emp-inc,...,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not in relationship,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native.country_United-States
0,90,11.252262,40,0,8.379539,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
1,82,11.797134,18,0,8.379539,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
2,66,12.133835,40,0,8.379539,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
3,54,11.851966,40,0,8.268988,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
4,41,12.486216,40,0,8.268988,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1


In [6]:
X = dummies_data.drop(['income'], axis=1)
y = dummies_data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.7


### Задание 3.5

Реализуйте настройку гиперпараметров алгоритма RandomForestClassifier(random_state=42) со следующей сеткой значений:

n_estimators = от 100 до 300 включительно с шагом 10
min_samples_leaf = от 3 до 7 с шагом 1
max_depth = от 15 до 40 с шагом 1
Используйте Hyperopt с параметрами max_evals = 20.

In [8]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
      }

In [9]:
# зафиксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train_scaled, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [16]:
# %%time # начинаем подбор гиперпараметров


trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space = space, # пространство гиперпараметров
          algo = tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals = 20, # максимальное количество итераций
          trials = trials, # логирование результатов
          # rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
          rstate = np.random.default_rng(random_state)
         )
print("Наилучшие значения гиперпараметров {}".format(best))



100%|██████████| 20/20 [01:28<00:00,  4.40s/trial, best loss: -0.7639828998931244]
Наилучшие значения гиперпараметров {'max_depth': 30.0, 'min_samples_leaf': 3.0, 'n_estimators': 130.0}


In [17]:
model = ensemble.RandomForestClassifier(random_state=random_state, n_estimators=int(best['n_estimators']),   max_depth=int(best['max_depth']),min_samples_leaf=int(best['min_samples_leaf']))
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.68


In [18]:
# # зададим пространство поиска гиперпараметров
# space={'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
#        'max_depth' : hp.quniform('max_depth', 15, 40, 1),
#        'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
#       }
# # зафксируем random_state
# random_state = 42
# def hyperopt_gb(params, cv=5, X=X_train_scaled, y=y_train, random_state=random_state):
#     # функция получает комбинацию гиперпараметров в "params"
#     params = {'n_estimators': int(params['n_estimators']), 
#               'max_depth': int(params['max_depth']), 
#               'min_samples_leaf': int(params['min_samples_leaf'])
#               }
#     # используем эту комбинацию для построения модели
#     model = ensemble.RandomForestClassifier(**params, random_state=random_state)
#  
#     # обучаем модель
#     model.fit(X, y)
#     score = metrics.f1_score(y, model.predict(X))
#     # метрику необходимо минимизировать, поэтому ставим знак минус
#     return -score
#  
#     # начинаем подбор гиперпараметров
# best=fmin(hyperopt_gb, # наша функция 
#           space=space, # пространство гиперпараметров
#           algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
#           max_evals=20, # максимальное количество итераций
#           trials=trials, # логирование результатов
#           # rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
#           rstate = np.random.default_rng(random_state)
#          )
#  
# # рассчитаем точность для тестовой выборки
# model = ensemble.RandomForestClassifier(random_state=random_state, n_estimators=int(best['n_estimators']),   max_depth=int(best['max_depth']),min_samples_leaf=int(best['min_samples_leaf']))
# model.fit(X_train_scaled, y_train)
# y_test_pred = model.predict(X_test_scaled)
# print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

### Задание 3.10

Реализуйте настройку гиперпараметров алгоритма RandomForestClassifier(random_state=42) со следующей сеткой значений:

n_estimators = от 100 до 300 включительно с шагом 10
min_samples_leaf = от 3 до 7 с шагом 1
max_depth = от 15 до 40 с шагом 1
Используйте Optuna с параметрами n_trails = 20.

В ответе укажите метрику f1 на тестовой выборке, значение округлите до двух знаков после запятой (например, 0.58).

In [19]:
import optuna

print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.0.2


In [20]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 10)
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train_scaled, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train_scaled))

  return score

In [21]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-10-05 22:45:56,965] A new study created in memory with name: RandomForestClassifier
[I 2022-10-05 22:46:00,033] Trial 0 finished with value: 0.711710071025314 and parameters: {'n_estimators': 130, 'max_depth': 30, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.711710071025314.
[I 2022-10-05 22:46:04,899] Trial 1 finished with value: 0.7041225829989054 and parameters: {'n_estimators': 210, 'max_depth': 37, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.711710071025314.
[I 2022-10-05 22:46:09,089] Trial 2 finished with value: 0.7020730141258484 and parameters: {'n_estimators': 190, 'max_depth': 20, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.711710071025314.
[I 2022-10-05 22:46:14,256] Trial 3 finished with value: 0.7387516838796586 and parameters: {'n_estimators': 210, 'max_depth': 32, 'min_samples_leaf': 4}. Best is trial 3 with value: 0.7387516838796586.
[I 2022-10-05 22:46:19,461] Trial 4 finished with value: 0.7087816091954022 and parameters: {'n_esti

CPU times: total: 1min 47s
Wall time: 1min 47s


In [22]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))


Наилучшие значения гиперпараметров {'n_estimators': 270, 'max_depth': 27, 'min_samples_leaf': 3}
f1_score на обучающем наборе: 0.76


In [23]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test_scaled, y_test)))
y_test_pred = model.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.86
f1_score на тестовом наборе: 0.68
